In [2]:
%load_ext cython

In [8]:
%%cython -a

import cython
import numpy as np
cimport numpy as np

DTYPE = np.float
ctypedef np.float_t DTYPE_t

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def normalized_cc(np.ndarray[DTYPE_t, ndim=2] s1, np.ndarray[DTYPE_t, ndim=2] s2,
                  float norm1=-1., float norm2=-1.):
    assert s1.dtype == DTYPE and s2.dtype == DTYPE
    assert s1.shape[1] == s2.shape[1]
    cdef DTYPE_t s = 0.
    cdef int sz = s1.shape[0]
    cdef int d = s1.shape[1]
    # Compute fft size based on tip from
    # https://stackoverflow.com/questions/14267555/how-can-i-find-the-smallest-power-of-2-greater-than-n-in-python
    cdef int fft_sz = 1 << bit_length(2 * sz - 1)
    cdef float denom = 0.
    cdef np.ndarray[DTYPE_t, ndim=2] cc

    if norm1 < 0.:
        norm1 = np.linalg.norm(s1)
    if norm2 < 0.:
        norm2 = np.linalg.norm(s2)

    denom = norm1 * norm2
    if denom < 1e-9:  # To avoid NaNs
        denom = np.inf

    cc = np.real(np.fft.ifft(np.fft.fft(s1, fft_sz, axis=0) *
                                   np.conj(np.fft.fft(s2, fft_sz, axis=0)), axis=0))
    cc = np.vstack((cc[-(sz-1):], cc[:sz]))
    return np.real(cc).sum(axis=-1) / denom


@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def cdist_normalized_cc(np.ndarray[DTYPE_t, ndim=3] dataset1, np.ndarray[DTYPE_t, ndim=3] dataset2,
                        np.ndarray[DTYPE_t, ndim=1] norms1, np.ndarray[DTYPE_t, ndim=1] norms2,
                        bool self_similarity):
    assert dataset1.dtype == DTYPE and dataset2.dtype == DTYPE
    assert dataset1.shape[2] == dataset2.shape[2]
    cdef int i = 0
    cdef int j = 0
    cdef np.ndarray[DTYPE_t, ndim=2] dists = np.empty((dataset1.shape[0], dataset2.shape[0]))

    if (norms1 < 0.).any():
        norms1 = np.linalg.norm(dataset1, axis=(1, 2))
    if (norms2 < 0.).any():
        norms2 = np.linalg.norm(dataset2, axis=(1, 2))

    for i in range(dataset1.shape[0]):
        for j in range(dataset2.shape[0]):
            if self_similarity and j < i:
                dists[i, j] = dists[j, i]
            elif self_similarity and i == j:
                dists[i, j] = 0.
            else:
                dists[i, j] = normalized_cc(dataset1[i], dataset2[j], norm1=norms1[i], norm2=norms2[j]).max()
    return dists


Error compiling Cython file:
------------------------------------------------------------
...

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def cdist_normalized_cc(np.ndarray[DTYPE_t, ndim=3] dataset1, np.ndarray[DTYPE_t, ndim=3] dataset2,
                        np.ndarray[DTYPE_t, ndim=1] norms1, np.ndarray[DTYPE_t, ndim=1] norms2,
                        bool self_similarity):
                       ^
------------------------------------------------------------

/Users/navaro/.ipython/cython/_cython_magic_febb750cad3ed5f187861816745f9387.pyx:43:24: 'bool' is not a type identifier

Error compiling Cython file:
------------------------------------------------------------
...
    cdef DTYPE_t s = 0.
    cdef int sz = s1.shape[0]
    cdef int d = s1.shape[1]
    # Compute fft size based on tip from
    # https://stackoverflow.com/questions/14267555/how-can-i-find-the-smallest-